# EX6-STRUCT: SparkSQL basics

Your assignment: complete the `TODO`'s and include also the **output of each cell**.

#### You may need to read the [Dataframe API Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html) to complete this lab.

### Step 1: Download Bike Trip Data (GDrive)

In [2]:
import os
import gdown

# stations data
if not os.path.isfile('data/station-data.json'):
    id = "134kLURYaTZuj6SWrg1XvkQKQ30bf1X7I"
    gdown.download(id=id, output="data/")
    print("Stations data downloaded")
else:
    print("Stations data already downloaded")

# trips data
if not os.path.isfile('data/trip-data.json'):
    id = "1pX3WHi3R2n52zyo6swXxlOtVTgt_hOQG"top30_rides = trips_df.orderBy(col("duration").desc()).limit(30)

top30_with_coords = top30_rides.join(
    stations_df,
    top30_rides["start_station_name"] == stations_df["name"],
    how="left"
).select(
    top30_rides["start_station_name"].alias("station"),
    stations_df["latitude"].alias("lat"),
    stations_df["longitude"].alias("long"),
    top30_rides["duration"].alias("ride_duration")
)

top30_with_coords.show(30, truncate=False)
    gdown.download(id=id, output="data/")
    print("Trips data downloaded")
else:
    print("Trips data already downloaded")

SyntaxError: invalid syntax (3507536760.py, line 14)

### Step 2: Start Spark Session

In [3]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except NameError:
    print("SparkContext not defined")

# local mode
spark = SparkSession.builder \
            .appName("Spark SQL basic example") \
            .master("local[*]") \
	    	.config("spark.some.config.option", "some-value") \
	    	.getOrCreate()

# cluster mode
#spark = SparkSession.builder \
#            .appName("Spark SQL basic example") \
#            .master("spark://spark:7077") \
#	    	.config("spark.some.config.option", "some-value") \
#	    	.getOrCreate()

SparkContext not defined


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 00:48:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 3: Read JSON files as dataframes

In [4]:
stations_df = spark.read.json('data/station-data.json')
stations_df.show()

+---------+------------+------------+---------+-----------+--------------------+----------+
|dockcount|installation|    landmark|      lat|       long|                name|station_id|
+---------+------------+------------+---------+-----------+--------------------+----------+
|       27|  2013-08-06|    San Jose|37.329732|-121.901782|San Jose Diridon ...|         2|
|       15|  2013-08-05|    San Jose|37.330698|-121.888979|San Jose Civic Ce...|         3|
|       11|  2013-08-06|    San Jose|37.333988|-121.894902|Santa Clara at Al...|         4|
|       19|  2013-08-05|    San Jose|37.331415|  -121.8932|    Adobe on Almaden|         5|
|       15|  2013-08-07|    San Jose|37.336721|-121.894074|    San Pedro Square|         6|
|       15|  2013-08-07|    San Jose|37.333798|-121.886943|Paseo de San Antonio|         7|
|       15|  2013-08-05|    San Jose|37.330165|-121.885831| San Salvador at 1st|         8|
|       15|  2013-08-05|    San Jose|37.348742|-121.894715|           Japantown|

In [5]:
trips_df = spark.read.json('data/trip-data.json').repartition(9)
trips_df.show()

[Stage 3:============================>                           (12 + 12) / 24]

+-------+--------+--------------------+--------------------+------------+--------------------+--------------------+--------------+---------------+-------+--------+
|bike_id|duration|            end_date|         end_station|end_terminal|          start_date|       start_station|start_terminal|subscriber_type|trip_id|zip_code|
+-------+--------+--------------------+--------------------+------------+--------------------+--------------------+--------------+---------------+-------+--------+
|    553|     698|2015-08-26T00:09:...|     Spear at Folsom|          49|2015-08-26T00:08:...|San Francisco Cal...|            69|     Subscriber| 905821|   94040|
|    292|     628|2015-08-20T00:09:...|  Powell Street BART|          39|2015-08-20T00:09:...|     Townsend at 7th|            65|     Subscriber| 898176|   94107|
|    506|     503|2015-08-28T00:08:...|       2nd at Folsom|          62|2015-08-28T00:08:...|Harry Bridges Pla...|            50|     Subscriber| 909314|   94558|
|    520|     44

### Step 4: Use operations `select`, `distinct`, and `count` to determine "how many different zip codes exists in trips data" `#TODO`

In [6]:
from pyspark.sql.functions import col

zip_count = trips_df.select(col("zip_code")).distinct().count()
print(f"Number of different zip codes: {zip_count}")

Number of different zip codes: 4705


### Step 5: Dataframe = RDD with schema
Observe that a dataframe is an RDD with each item being of type `Row`. Naturally, rows have schema associated with.

In [7]:
stations_df.take(3)

[Row(dockcount=27, installation='2013-08-06', landmark='San Jose', lat=37.329732, long=-121.901782, name='San Jose Diridon Caltrain Station', station_id=2),
 Row(dockcount=15, installation='2013-08-05', landmark='San Jose', lat=37.330698, long=-121.888979, name='San Jose Civic Center', station_id=3),
 Row(dockcount=11, installation='2013-08-06', landmark='San Jose', lat=37.333988, long=-121.894902, name='Santa Clara at Almaden', station_id=4)]

In [8]:
stations_df.printSchema()

root
 |-- dockcount: long (nullable = true)
 |-- installation: string (nullable = true)
 |-- landmark: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- name: string (nullable = true)
 |-- station_id: long (nullable = true)



### Step 6: Extract from trips data the top 10 latest rides -- the ones that happened more recently

In [9]:
trips_df.printSchema()

root
 |-- bike_id: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station: string (nullable = true)
 |-- end_terminal: long (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station: string (nullable = true)
 |-- start_terminal: long (nullable = true)
 |-- subscriber_type: string (nullable = true)
 |-- trip_id: long (nullable = true)
 |-- zip_code: long (nullable = true)



In [10]:
# start_date as string
trips_df.select(col("start_date")).first()

Row(start_date='2015-08-17T00:19:06.000-03:00')

In [11]:
from pyspark.sql.functions import to_date, col, unix_timestamp, to_timestamp

# new column with start_date formatted as datetime (now we have the notion of 'order' between dates)
trips_df = trips_df.withColumn("start_date_fmt", to_timestamp(col("start_date"), "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"))

In [12]:
top10_latest_rides = trips_df.orderBy(col("start_date_fmt").desc()).limit(10)
top10_latest_rides.show(truncate=False)

+-------+--------+-----------------------------+---------------------------------------------+------------+-----------------------------+---------------------------------------------+--------------+---------------+-------+--------+-------------------+
|bike_id|duration|end_date                     |end_station                                  |end_terminal|start_date                   |start_station                                |start_terminal|subscriber_type|trip_id|zip_code|start_date_fmt     |
+-------+--------+-----------------------------+---------------------------------------------+------------+-----------------------------+---------------------------------------------+--------------+---------------+-------+--------+-------------------+
|288    |765     |2015-08-31T00:23:39.000-03:00|San Francisco Caltrain (Townsend at 4th)     |70          |2015-08-31T00:23:26.000-03:00|Harry Bridges Plaza (Ferry Building)         |50            |Subscriber     |913460 |2139    |2015-08-31 03

### Step 7: Get the latitude and longitude of the **starting stations** that produced the top-30 longest rides `#TODO`

Generate a list such as:
```
station1 lat long ride_duration
station2 lat long ride_duration
station3 lat long ride_duration
...
```

Obs: the same station may appear more than once in this list

In [14]:
top30_rides = trips_df.orderBy(col("duration").desc()).limit(30)

top30_with_coords = top30_rides.join(
    stations_df,
    top30_rides["start_station"] == stations_df["name"],
    how="left"
).select(
    top30_rides["start_station"].alias("station"),
    stations_df["lat"].alias("lat"),
    stations_df["long"].alias("long"),
    top30_rides["duration"].alias("ride_duration")
)

top30_with_coords.show(30, truncate=False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `latitude` cannot be resolved. Did you mean one of the following? [`dockcount`, `installation`, `landmark`, `lat`, `long`, `name`, `station_id`].